In [5]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

#Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines

print("Repositories uploaded!!")

Repositories uploaded!!


In [6]:
########################################################################################################################################################
ConcreteData_df = pd.read_excel('UCI_regression/Concrete_Data.xls') ## 'Cement' found to be correlated at 0.4 :: 100
# HousingData_df = pd.read_csv('UCI_regression/BostonHousing/BostonHousing.csv') ## 'nox' found to be correlated at 0.4 :: [0.385 - 0.871] :: 50
# dropcol_initial = ['name']
# AutoData_df = pd.read_csv('UCI_regression/MPG/Auto.csv') ## horsepower column has correlation 0.4 :: [46 - 230] :: 30
# AutoData_df = AutoData_df.drop(dropcol_initial, axis = 1)
# print("The shape of the Input data is: ", AutoData_df.shape)

############################# Preprocessing Data ################################################################
## Finding the correlation first
print("The correlation matrix is: ")
print(ConcreteData_df.corr())

col = ConcreteData_df['Cement']
ConcreteData_df = ConcreteData_df.sort_values(by=['Cement'])
# print(AutoData_df[(col <= 80)].shape)
# print(AutoData_df[(col > 80) & (col <= 110)].shape)
# print(AutoData_df[(col > 110)].shape)

# col = HousingData_df['nox']
# print(HousingData_df[(col < 0.450)].shape)
# print(HousingData_df[(col > 0.450) & (col < 0.600)].shape)


drop_col = ['Cement']
# drop_col = ['horsepower']
# drop_col = ['nox']

Train_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] <= 225)]
# Train_df = HousingData_df.loc[(HousingData_df['nox'] > 0.475) & (HousingData_df['nox'] <= 0.600)]
# Train_df = AutoData_df.loc[(AutoData_df['horsepower'] > 80) & (AutoData_df['horsepower'] <= 110)]
Train_df = Train_df.drop(drop_col, axis = 1)
Train_df = Train_df.reset_index(drop=True)
print(Train_df.shape)

Source_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 225) & (ConcreteData_df['Cement'] <= 350)]
# Source_df = HousingData_df.loc[(HousingData_df['nox'] <= 0.475)]
# Source_df = AutoData_df.loc[(AutoData_df['horsepower'] <= 80)]
Source_df = Source_df.drop(drop_col, axis = 1)
Source_df = Source_df.reset_index(drop=True)
print(Source_df.shape)

Test_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 350)]
# Test_df = HousingData_df.loc[(HousingData_df['nox'] > 0.600)]
# Test_df = AutoData_df.loc[(AutoData_df['horsepower'] > 110)]
Test_df = Test_df.drop(drop_col, axis = 1)
Test_df = Test_df.reset_index(drop=True)
print(Test_df.shape)


The correlation matrix is: 
                               Cement  BlastFurnaceSlag    FlyAsh     Water  \
Cement                       1.000000         -0.275193 -0.397475 -0.081544   
BlastFurnaceSlag            -0.275193          1.000000 -0.323569  0.107286   
FlyAsh                      -0.397475         -0.323569  1.000000 -0.257044   
Water                       -0.081544          0.107286 -0.257044  1.000000   
Superplasticizer             0.092771          0.043376  0.377340 -0.657464   
CoarseAggregate             -0.109356         -0.283998 -0.009977 -0.182312   
FineAggregate               -0.222720         -0.281593  0.079076 -0.450635   
Age                          0.081947         -0.044246 -0.154370  0.277604   
ConcreteCompressiveStrength  0.497833          0.134824 -0.105753 -0.289613   

                             Superplasticizer  CoarseAggregate  FineAggregate  \
Cement                               0.092771        -0.109356      -0.222720   
BlastFurnaceSlag   

In [ ]:
# ######################### Finding best instances from the source dataset ################################################################
# Source_df["ManDis"] = ""
#
# train_df_mean = []
# prow = Train_df.mean()
# train_df_mean = [prow.BlastFurnaceSlag, prow.FlyAsh, prow.Water, prow.Superplasticizer, prow.CoarseAggregate, prow.FineAggregate, prow.Age, prow.ConcreteCompressiveStrength]
# # train_df_mean = [prow.crim, prow.zn, prow.indus, prow.chas, prow.rm, prow.age, prow.dis, prow.rad, prow.tax, prow.ptratio, prow.b, prow.lstat, prow.medv]
# # train_df_mean = [prow.mpg, prow.cylinders, prow.displacement, prow.weight, prow.acceleration, prow.year, prow.origin]
#
# rowidx = 0
# for row in Source_df.itertuples():
#     row_list =[row.BlastFurnaceSlag, row.FlyAsh, row.Water, row.Superplasticizer, row.CoarseAggregate, row.FineAggregate, row.Age, row.ConcreteCompressiveStrength]
#     # row_list =[row.crim, row.zn, row.indus, row.chas, row.rm, row.age, row.dis, row.rad, row.tax, row.ptratio, row.b, row.lstat, row.medv]
#     # row_list =[row.mpg, row.cylinders, row.displacement, row.weight, row.acceleration, row.year, row.origin]
#
#     man_dis = 0
#     for i in range(0, len(row_list)):
#         tempval = train_df_mean[i] - row_list[i]
#         man_dis = man_dis + abs(tempval)
#
#     Source_df.loc[rowidx,"ManDis"] = man_dis
#     # print(Source_df.loc[rowidx,"ManDis"])
#     rowidx = rowidx + 1
#
# # print(Source_df)
# Source_df = Source_df.sort_values('ManDis')
# Train_source_df = Source_df.head(100) ## For housing 70 was taken, For auto 40 was taken, For concrete 100 was taken
# Source_df = Source_df.iloc[100:]
# Source_df = Source_df.drop(['ManDis'], axis =1)
# Train_source_df = Train_source_df.drop(['ManDis'], axis =1)
#
# # print(Train_source_df.shape)
# # print(Source_df.shape)

In [8]:
################################################################################################################################################################

target_column = ['ConcreteCompressiveStrength']
# target_column = ['medv']
# target_column = ['mpg']

Train_df = pd.concat([Train_df, Source_df], ignore_index=True)

Train_df_y = Train_df[target_column]
Train_df_X = Train_df.drop(target_column, axis = 1)

Test_df_y = Test_df[target_column]
Test_df_X = Test_df.drop(target_column, axis = 1)

Source_df_y = Source_df[target_column]
Source_df_X = Source_df.drop(target_column, axis = 1)

print(Train_df_X.shape)
print(Test_df_X.shape)
print(Source_df_X.shape)

## Merging the datasets
X_df = pd.concat([Train_df_X, Test_df_X], ignore_index=True)
y_df = pd.concat([Train_df_y, Test_df_y], ignore_index=True)

np_train_X = X_df.to_numpy()
np_train_y = y_df.to_numpy()

np_source_X = Source_df_X.to_numpy()
np_source_y = Source_df_y.to_numpy()

np_train_y_list = np_train_y.ravel()

# sample_size = [len(Target_df_X), len(Source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)


(774, 7)
(256, 7)
(406, 7)


In [10]:
################################################TwoStageAdaBoostR2############################################################################

kf = KFold(n_splits = 2) ## Create no. of CV Folds
error = []

for train_idx, test_idx in kf.split(X_df):
    X_train, X_test = np_train_X[train_idx], np_train_X[test_idx]
    y_train, y_test = np_train_y[train_idx], np_train_y[test_idx]

    sample_size = [len(Source_df_X), len(X_train)]

    X_train_new = np.concatenate((np_source_X, X_train))
    y_train_new = np.concatenate((np_source_y, y_train))

    np_y_train_new_list = y_train_new.ravel()

    regr_1 = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth = 6),
                      n_estimators = n_estimators, sample_size = sample_size,
                      steps = steps, fold = fold,
                      random_state = random_state)
    regr_1.fit(X_train_new, np_y_train_new_list)
    y_pred2 = regr_1.predict(X_test)
    mse_adaboost = sqrt(mean_squared_error(y_test, y_pred2))
    print("RMSE of updated TrAdaboost.R2:", mse_adaboost)
    error.append(mse_adaboost)

print("Mean RMSE: ", sum(error)/len(error))


RMSE of updated TrAdaboost.R2: 13.304089493657921
RMSE of updated TrAdaboost.R2: 14.91072696784535
Mean RMSE:  14.107408230751634
